In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY") 



In [3]:
chat = ChatOpenAI(
    openai_api_key=openai_key, 
    model='gpt-3.5-turbo'
)

In [4]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

In [5]:
res = chat(messages)

res

AIMessage(content='String theory is a theoretical framework in physics that aims to explain the fundamental nature of matter and the forces that govern it. It suggests that instead of particles being point-like, they are actually tiny, vibrating strings. These strings can vibrate in different modes, and each mode corresponds to a different particle with specific properties such as mass and charge.\n\nOne of the key features of string theory is that it attempts to unify all the known fundamental forces of nature: gravity, electromagnetism, and the strong and weak nuclear forces. It suggests that these forces are all different manifestations of the vibrations of these tiny strings.\n\nString theory also proposes the existence of additional dimensions beyond the three spatial dimensions (length, width, and height) that we are familiar with. These extra dimensions, if they exist, are believed to be curled up and too small to be detected directly.\n\nIt is important to note that string theo

In [6]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it incorporates all the fundamental forces of nature within a single framework. In other words, it aims to provide a consistent description of how gravity, electromagnetism, and the strong and weak nuclear forces are connected and operate at a fundamental level.

One of the main motivations for seeking a unified theory is the desire to resolve the incompatibility between quantum mechanics, which describes the behavior of particles on very small scales, and general relativity, which describes the behavior of gravity on large scales. These two theories are extremely successful in their respective domains, but they have different mathematical formulations and cannot be easily combined.

String theory offers a potential solution by replacing point-like particles with tiny, vibrating strings. This fundamentally different approach allows for the possibility of unifying the forces of nature. The vibrat

loading the challenge document 

In [7]:
import PyPDF2

# Open the PDF file in binary read mode
with open("/home/hp/Documents/week 5/precision-RAG-prompt-tuning/prompt_generation_system/10 Academy Cohort A - Weekly Challenge Week - 6.pdf", "rb") as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)

    num_pages = len(pdf_reader.pages)

    for page in range(num_pages):
        page_obj = pdf_reader.pages[page]
        extracted_text = page_obj.extract_text()
        
len(extracted_text)

511

In [8]:
chunk_size=50
overlap=20
def chunk_text(extracted_text, chunk_size, overlap):
    chunks = []
    text_length = len(extracted_text)

    for start in range(0, text_length, chunk_size - overlap):
        end = min(start + chunk_size, text_length)
        chunk = extracted_text[start:end]
        chunks.append(chunk)

    return chunks

chunks = chunk_text(extracted_text, chunk_size, overlap)

for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:", chunk)

Chunk 1: References
●
Meistrari
didn’t
see
a
good
solution

Chunk 2: see
a
good
solution
for
prompt
engineering,
so
it’
Chunk 3: 
engineering,
so
it’s
building
one
●
AutoPrompt:
E
Chunk 4: 
one
●
AutoPrompt:
Eliciting
Knowledge
from
Langua
Chunk 5: nowledge
from
Language
Models
with
Automatically
G
Chunk 6: with
Automatically
Generated
Prompts
●
Large
Langu
Chunk 7: rompts
●
Large
Language
Models
Are
Human-Level
Pro
Chunk 8: 
Are
Human-Level
Prompt
Engineers
●
Prompt
Enginee
Chunk 9: ers
●
Prompt
Engineering
●
How
to
Create
a
Monte
C
Chunk 10: 
to
Create
a
Monte
Carlo
Simulation
using
Python
●
Chunk 11: ation
using
Python
●
Monte
Carlo
Method
Explained

Chunk 12: lo
Method
Explained
●
What
is
Monte
Carlo
Simulati
Chunk 13: Monte
Carlo
Simulation?
How
does
it
work?
●
Elo
Ra
Chunk 14: es
it
work?
●
Elo
Rating
Algorithm
●
Elo
algorithm
Chunk 15: ithm
●
Elo
algorithm
implementation
in
Python
●
Tr
Chunk 16: ation
in
Python
●
TrueSkillTM:
A
Bayesian
skill
ra
Chunk 17: 
A
Bayesian
skill
ratin

In [9]:
chunks = chunk_text(extracted_text, chunk_size, overlap)

#for i, chunk in enumerate(chunks):
    #print(f"Chunk {i + 1}:", chunk)
type(chunks)

list

initialize the client and connect to the targetted index 

In [15]:
import pinecone

api_key = os.getenv("api_keyy")
print(api_key)
# Initialize Pinecone with the API key
pinecone.init(api_key=api_key)

# Create or connect to the 'quickstart' index
index = pinecone.Index(index_name="mekdes-index")
# get API key from app.pinecone.io and environment from console
pinecone.init(
    api_key=api_key,
    environment='gcp-starter'
)

26b33591-e6c6-499a-b91a-37c5f40265df


AttributeError: module 'pinecone' has no attribute 'init'

In [11]:
import os
import pinecone

api_key = os.getenv("api_key")
print(api_key)





26b33591-e6c6-499a-b91a-37c5f40265df


AttributeError: module 'pinecone' has no attribute 'init'

In [ ]:
index.describe_index_stats('mekdes-index')

: 

embed and add chunks to pinecone index 

In [ ]:

import tensorflow as tf
import tensorflow_hub as hub

# Function to embed and add chunks to Pinecone index
def embed_and_add_to_pinecone(chunks, index_name):
    for i, chunk in enumerate(chunks):
        def embedding_function(text_chunk):
            # Load the pre-trained Universal Sentence Encoder model
            embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

            # Embed the text chunk
            embedded_chunk = embed([text_chunk])[0].numpy().tolist()

            return embedded_chunk

        # Embed each chunk using your embedding method (not specified in the question)
        # Replace 'embedding_function' with the actual function you use for embedding
        embedding = embedding_function(chunk)

        # Add the embedded chunk to the Pinecone index using upsert
        pinecone.upsert(index_name, [{'id': str(i), 'vector': embedding}])


 

: 

In [ ]:
# Example usage
chunks = ["This is a sample text.", "Another example sentence."]
index_name = "your_index_name"

# Embed and add chunks to Pinecone
embed_and_add_to_pinecone(chunks, index_name)

# Close Pinecone connection
pinecone.deinit()

: 

: 

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Load a pre-trained Word2Vec model (download and load your specific model)
# For example, you might use a model trained on a large text corpus
# This is just a placeholder, and you need to replace it with your actual model loading logic
word2vec_model = Word2Vec.load("path/to/your/word2vec_model")

def embed_function(chunk):
    # Tokenize the text (replace this with the appropriate tokenization for your data)
    tokens = word_tokenize(chunk.lower())
    
    # Embed each token and calculate the average vector as the representation of the chunk
    vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    
    if vectors:
        # Calculate the average vector
        average_vector = sum(vectors) / len(vectors)
        return average_vector
    else:
        # Return a placeholder vector if no valid tokens are found
        return [0.0] * word2vec_model.vector_size

# Example usage:
chunk_data = ["This is a sample text.", "Another example sentence."]
chunk_ids = [1, 2]

vectors = [embed_function(chunk) for chunk in chunk_data]

# Assuming you already created a Pinecone index named 'your_index_name'
index.upsert(ids=chunk_ids, vectors=vectors)


: 